In [ ]:
import os
from google.colab import drive

# Connect to Google Drive
drive.mount('/content/gdrive')

# Go to the root drive folder
os.chdir('/content/gdrive/MyDrive/INASCON2021')


In [1]:
# import some stuff

import math
from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
#from tqdm import tqdm
from tqdm.notebook import tqdm

In [7]:
# This function computes a scattering profile from an image
# assuming on average the orientation doesn't matter.
# Typically, we would do an orientation-average on a 2d FFT
# result. For a package that does that, look at PyFAI.
# Here we assume the orientation doesn't matter, so we "cheat"
# and average the Fourier spectrum on the vertical and
# horizontal directions
def SimplifiedScatteringProfile(image):
    # Perform one-dimensional fft over the first axis
    f = np.fft.fft(image, axis=0)

    # Calculate the magnitude and phase spectra
    magnitude_spectrum0 = np.abs(f)
    magnitude_spectrum0 = np.sum(magnitude_spectrum0, 1)
    magnitude_spectrum0 = magnitude_spectrum0[0:magnitude_spectrum0.shape[0]//2]

    #return magnitude_spectrum0

    # Perform one-dimensional fft over the second axis
    f = np.fft.fft(image, axis=1)

    # Calculate the magnitude and phase spectra
    magnitude_spectrum1 = np.abs(f)
    magnitude_spectrum1 = np.sum(magnitude_spectrum1, 0)
    magnitude_spectrum1 = magnitude_spectrum1[0:magnitude_spectrum1.shape[0]//2]

    return 0.5 * (magnitude_spectrum0 + magnitude_spectrum1)


In [112]:
def ComputeVideoScatteringProfile(filepath, L):
    scattering_profile_list = []
    frame_bw = []

    vid = cv.VideoCapture(filepath)

    number_of_frames = vid.get(cv.CAP_PROP_FRAME_COUNT)
    width = vid.get(cv.CAP_PROP_FRAME_WIDTH)

    progressbar = tqdm(total=number_of_frames)
    count = 0

    while vid.isOpened():
        ret, frame = vid.read()  # read a frame

        # No frame? end the loop
        if not ret:
            break

        count += 1

        # convert frame to grayscale and then to 0/1
        frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        frame_bw = np.greater(frame, 0)  # > 0

        # Calculate scattering profile
        scattering_profile = SimplifiedScatteringProfile(frame)
        scattering_profile_list.append(scattering_profile)
        progressbar.update()

        # Uncomment if you want a quicker (and less accurate) result
        #if count == 100:
            #break

    vid.release()
    progressbar.close()

    # Average over scattering profiles of all frames
    scattering_profile = np.mean(scattering_profile_list, axis=0)

    # Generate the transfer-momentum (frequency) scale
    freqs = np.fft.fftfreq(len(scattering_profile) * 2, L / width)
    freqs = freqs[0:len(freqs) // 2]
    q = 2 * math.pi * freqs

    return scattering_profile, q, frame_bw

In [ ]:
(scattering_profile, q, frame) = ComputeVideoScatteringProfile('scattering_data/bots_flipflop_L1.1_N10_every10.mp4', 1.1)

# Plot
fig = plt.figure()
plt.imshow(frame, cmap='gray')
plt.title('Last Frame'), plt.xticks([]), plt.yticks([])
# plt.colorbar()
plt.show()


fig = plt.figure()
plt.loglog(q, scattering_profile)
plt.title('Scattering profile')
plt.show()
